In [ ]:
import numpy as np
import itertools
from scipy.optimize import minimize

def generate_beta(n, r, M):
    beta = {s: np.random.uniform(-M, M, n) for s in range(2, r + 1)}
    return beta

def generate_r_layered_hypergraph(n, r, beta):
    
    hyperedges = {}
    degrees = {s: np.zeros(n, dtype=int) for s in range(2, r + 1)}
    
    for s in range(2, r + 1):
        beta_s = beta[s]
        edges = []
        for comb in itertools.combinations(range(n), s):
            weight_sum = np.sum(beta_s[list(comb)])
            prob = np.exp(weight_sum) / (1 + np.exp(weight_sum))
            if np.random.rand() < prob:
                edges.append(comb)
                for v in comb:
                    degrees[s][v] += 1
        hyperedges[s] = edges
    return hyperedges, degrees

def add_gaussian_noise(degrees, epsilon, delta, r):
    noisy_degrees = {}
    c_squared = 2 * np.log(1.25 / delta)
    sigma = np.sqrt(c_squared) * np.sqrt(r) / epsilon
    
    for s, d_s in degrees.items():
        noise = np.random.normal(0, sigma, size=d_s.shape)
        noisy_degrees[s] = d_s + noise
    return noisy_degrees

def gradient_descent_beta(n, s, noisy_degrees_s, lr=0.01, max_iter=500, tol=1e-6, verbose=True):
    
    beta = np.zeros(n)
    combs = np.array(list(itertools.combinations(range(n), s)))  # shape (C, s)
    n_edges = combs.shape[0]

    for t in range(max_iter):
        # Compute weight sums: shape (C,)
        beta_sums = np.sum(beta[combs], axis=1)

        # Stable sigmoid
        sigm = np.empty_like(beta_sums)
        pos = beta_sums >= 0
        neg = ~pos
        
        # For positive beta_sums
        sigm[pos] = 1 / (1 + np.exp(-beta_sums[pos]))
        
        # For negative beta_sums: sigmoid(x) = exp(x) / (1 + exp(x)) is unstable; use equivalent safe form
        exp_x = np.exp(beta_sums[neg])
        sigm[neg] = exp_x / (1 + exp_x)

        # Accumulate gradient: shape (n,)
        grad = np.zeros(n)
        for idx in range(s):
            np.add.at(grad, combs[:, idx], sigm)  # Each position accumulates

        # Final gradient step
        grad -= noisy_degrees_s

        grad_norm = np.linalg.norm(grad)
        if verbose and t % 50 == 0:
            print(f"Iter {t}: grad norm = {grad_norm:.4f}")

        if grad_norm < tol:
            break

        beta -= lr * grad

    return beta


def run_simulation(n, r, M, epsilon, delta):
    beta = generate_beta(n, r, M)
    hyperedges, degrees = generate_r_layered_hypergraph(n, r, beta)
    noisy_degrees = add_gaussian_noise(degrees, epsilon, delta, r)
    
    errors = {}
    for s in range(2, r + 1):
        beta_hat = gradient_descent_beta(n, s, noisy_degrees[s], lr=0.01, max_iter=1000)
        errors[s] = np.linalg.norm(beta[s] - beta_hat)
        print(f"  ℓ2 error for s = {s}: {errors[s]:.4f}")
    return errors

if __name__ == "__main__":
    n = 200         # number of nodes
    r = 3        # number of layers
    M = 1.0        # max absolute value of beta
    epsilon = 0.5  # privacy parameter
    delta = 1e-5   # privacy parameter
    seed = 42      # for reproducibility

    errors = run_simulation(n, r, M, epsilon, delta)